In [1]:
import sys
from pathlib import Path

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper
import tasks.preprocessing

In [2]:
def base_df():
    df = pd.read_csv(
        "../data/100_annotators.csv",
        converters={"annot_personality_characteristics": ast.literal_eval},
    )
    df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
    df["annot_politics"] = df.annot_personality_characteristics.apply(
        lambda x: x[0]
    )
    df.annot_age = pd.cut(df.annot_age, bins=4)
    df.message_id = df.message_id.astype(str)
    df["comment_key"] = df.conv_id + df.message_id

    df = df.loc[
        :,
        [
            "conv_id",
            "message_id",
            "comment_key",
            "message",
            "toxicity",
            "annot_age",
            "annot_sex",
            "annot_sexual_orientation",
            "annot_demographic_group",
            "annot_current_employment",
            "annot_education_level",
            "annot_politics",
        ],
    ]
    df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
        lambda x: pd.Series(
            {
                col: x[col].tolist()
                for col in df.columns
                if col not in ["conv_id", "message_id", "comment_key", "message"]
            }
        ),
        include_groups=False,
    )
    df = df.reset_index()
    return df

df = base_df()
df["random"] = tasks.preprocessing.get_rand_col(df, "annot_sex")


In [3]:
sdb_columns = [
    "annot_age",
    "annot_sex",
    "annot_demographic_group",
    "annot_sexual_orientation",
    "annot_current_employment",
    "annot_education_level",
    "annot_politics"
]
res = tasks.run_helper.run_all_results(
    df=df,
    sdb_columns=sdb_columns,
    value_col="toxicity",
    comment_key_col="comment_key",
)
res

Evaluating SDB dimensions:   0%|          | 0/7 [00:00<?, ?it/s]

0         1
sdb_column                                                          
annot_age                (14.927, 33.25]          0.009625  0.816667
                         (33.25, 51.5]            0.402084  0.816667
                         (51.5, 69.75]           -0.619121  0.816667
                         (69.75, 88.0]           -0.043931  0.816667
annot_current_employment blue-collar             -0.191895  0.800000
                         unemployed               0.639002  0.800000
                         white-collar            -0.324561  0.800000
annot_demographic_group  asian                    1.426481  0.816667
                         black                    0.549822  0.816667
                         other                   -2.662293  0.816667
                         white                    1.046674  0.816667
annot_education_level    high-school              0.918117  0.816667
                         none                    -0.938831  0.816667
                         university               0.262008  0.816667
annot_politics           apolitical              -2.296841  0.816667
                         left-wing liberal        1.055821  0.816667
                         right-wing conservative  0.683671  0.816667
annot_sex                female                  -0.224347  0.816667
                         male                    -0.477026  0.816667
                         non-binary               3.952309  0.816667
annot_sexual_orientation bisexual                 3.132816  0.883333
                         homosexual              -0.416731  0.883333
                         other                   -5.795435  0.883333
                         straight                 0.350611  0.883333

In [4]:
tasks.run_helper.run_result(
    df,
    sdb_column="random",
    value_col="toxicity",
    comment_key_col="comment_key",
)

,0,1
3,0.795462,0.816667
4,-1.423692,0.816667
1,0.092356,0.816667
2,0.120893,0.816667
